# Overlay Design Methodology

## Developing a simple adder IP using HLS

This IP was developed using HLS and adds two 32-bit integers together. The full code for the accelerator is:

```C++
void add(int a, int b, int& c) {
#pragma HLS INTERFACE ap_ctrl_none port=return
#pragma HLS INTERFACE s_axilite port=a
#pragma HLS INTERFACE s_axilite port=b
#pragma HLS INTERFACE s_axilite port=c

	c = a + b;
}
```

With a block diagram consisting solely of the HLS IP and required glue logic to connect it to the ZYNQ7 IP

![Simple Block Diagram](../images/attribute1.png)

To interact with the IP first we need to load the overlay containing the IP.

In [1]:
from pynq import Overlay

overlay = Overlay('/home/xilinx/overlay_tutorial/overlays/adder.bit')

All of the entries are accessible via attributes on the overlay class with the specified driver. Accessing the `scalar_add` attribute of the will create a driver for the IP - as there is no driver currently known for the `Add` IP core `DefaultIP` driver will be used so we can interact with IP core.

In [2]:
add_ip = overlay.scalar_add

In [3]:
add_ip.register_map

RegisterMap {
  a = Register(a=write-only),
  b = Register(b=write-only),
  c = Register(c=0),
  c_ctrl = Register(c_ap_vld=1, RESERVED=0)
}

We can interact with the IP using the register map directly

In [4]:
add_ip.register_map.a = 230
add_ip.register_map.b = 45
add_ip.register_map.c

Register(c=275)

## Creating a Driver

While the `DefaultIP` driver is useful for determining that the IP is working it is not the most user-friendly API to expose to the eventual end-users of the overlay. Ideally we want to create an IP-specific driver exposing a single `add` function to call the accelerator. Custom drivers are created by inheriting from `DefaultIP` and adding a `bindto` class attribute consisting of the IP types the driver should bind to. The constructor of the class should take a single `description` parameter and pass it through to the super class `__init__`. The description is a dictionary containing the address map and any interrupts and GPIO pins connected to the IP.

In [5]:
from pynq import DefaultIP

class AddDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
    
    bindto = ['xilinx.com:hls:add:1.0']
    
    def add(self, a, b):
        self.write(0x10, a)
        self.write(0x18, b)
        return self.read(0x20)

Now if we reload the overlay and query the help again we can see that our new driver is bound to the IP.

In [6]:
overlay = Overlay('/home/xilinx/overlay_tutorial/overlays/adder.bit')
overlay?

And we can access the same way as before except now our custom driver with an `add` function is created instead of `DefaultIP`

In [7]:
overlay.scalar_add.add(15,20)

35